In [15]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
import nltk

In [16]:
df = pd.read_csv(
    filepath_or_buffer='data/train.dat.txt', 
    header=None, 
    sep='\t')
df1 =  pd.read_csv(
    filepath_or_buffer='data/test.dat.txt', 
    header=None,
    sep='\n')
9
# separate names from classes
vals = df.ix[:,:].values
vals1 = df1.ix[:].values
#print vals
df.columns = ["cls","names"]
df1.columns = ["test"]

/Users/meenakshiparyani/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  # This is added back by InteractiveShellApp.init_path()
/Users/meenakshiparyani/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  if sys.path[0] == '':


In [17]:
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

df["names"] = [BeautifulSoup(text).get_text() for text in df["names"] ]
df["names"] = df["names"].map(lambda x: re.sub(r'\W+', " ", x))
df["names"] = df["names"].map(lambda x: re.sub(r'\d+', "", x))
df["names"] = df["names"].map(lambda x: WordNetLemmatizer().lemmatize(x))
df["names"] = df["names"].map(lambda x: PorterStemmer().stem(x))
df["names"] = df["names"].str.lower().str.split(" ")

stop = stopwords.words('english')
df["names"] = df["names"].apply(lambda x: [item for item in x if item not in stop])

df1["test"] = [BeautifulSoup(text).get_text() for text in df1["test"] ]
df1["test"] = df1["test"].map(lambda x: re.sub(r'\W+', " ", x))
df1["test"] = df1["test"].map(lambda x: re.sub(r'\d+', "", x))
df1["test"] = df1["test"].map(lambda x: WordNetLemmatizer().lemmatize(x))
df1["test"] = df1["test"].map(lambda x: PorterStemmer().stem(x))
df1["test"] = df1["test"].str.lower().str.split(" ")
df1["test"] = df1["test"].apply(lambda x: [item for item in x if item not in stop])

In [18]:
idx = {}
tid = 0
nnz = 0
def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    global idx
    global tid
    global nnz
    nrows = len(docs)
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
    return ncols,nrows

def matrices(ncols, nrows, docs):
        
    # set up memory
    global nnz
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    global idx
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
    
    mat = csr_matrix((val, ind, ptr), shape=(nrows, nnz), dtype=np.double)
    mat.sort_indices()
    return mat

ncols1, nrows1 = build_matrix(df["names"])
ncols2, nrows2 = build_matrix(df1["test"])
m1 = matrices(ncols1, nrows1, df["names"])
m2 = matrices(ncols2, nrows2, df1["test"])

In [19]:
def csr_l2normalize(mat,nnz, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    nrows = mat.shape[0]
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
    return csr_matrix((val, ind, ptr), shape=(nrows, nnz), dtype=np.double)
global n_m1
global n_m2
n_m1 = csr_l2normalize(m1, nnz)
n_m2 = csr_l2normalize(m2, nnz)

In [20]:
def classify(x, train, clstr, k=17):
        r""" Classify vector x using kNN and majority vote rule given training data and associated classes
        """
        # find nearest neighbors for x
        dots = x.dot(train.T)
        sims = list(zip(dots.indices, dots.data))
        if len(sims) == 0:
            # could not find any neighbors
            return '+' if np.random.rand() > 0.5 else '-'
        sims.sort(key=lambda x: x[1], reverse=True)
        tc = Counter(clstr[s[0]] for s in sims[:k]).most_common(2)
        if len(tc) < 2 or tc[0][1] > tc[1][1]:
            # majority vote
            return tc[0][0]
        # tie break
        tc = defaultdict(float)
        for s in sims[:k]:
            tc[clstr[s[0]]] += s[1]
        return sorted(tc.items(), key=lambda x: x[1], reverse=True)[0][0]

In [21]:
outputfile = open("output.dat", 'w')
l = [ classify(n_m2[i], n_m1, df["cls"]) for i in range(n_m2.shape[0]) ]
for i in range(len(l)):
    if str(l[i])=="1" or str(l[i])=="+1":
        outputfile.write("+1\n")
    if str(l[i])=="-1":
        outputfile.write("-1\n")
outputfile.close()  